This notebook add economic features and imputates data 

In [13]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('final_tuik.csv')

# Find columns with all empty values
empty_columns = df.columns[df.isnull().all()]

# Print the empty columns
print("Columns with all empty values:", empty_columns.tolist())

Columns with all empty values: ['tuitionFee', 'universityFoundingYear', 'facultyFoundingYear', 'revenue', 'tuitionFeeIncrease']


In [14]:

# Define the data for base salary, inflation, and growth by year
base_salary_by_year = {
    2019: 2103,
    2020: 2103,   
    2021: 2825,
    2022: 5500,
    2023: 11402,
    2024: 17002
}

inflation_by_year = {
    2019: 11.8,
    2020: 14,
    2021: 36,
    2022: 64,
    2023: 64,
    2024: 64
}

growth_by_year = {
    2019: 0.7,
    2020: 1.8,    
    2021: 11.6,
    2022: 5.4,
    2023: 3.8,
    2024: 2.7
}

# Open the CSV file
df = pd.read_csv('final_tuik.csv')

# Add the new columns based on the academic year
df['base_salary_by_year'] = df['academicYear'].map(base_salary_by_year)
df['inflation_by_year'] = df['academicYear'].map(inflation_by_year)
df['growth_by_year'] = df['academicYear'].map(growth_by_year)

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_final_tuik.csv', index=False)

# Display the updated DataFrame
df.head()

,academicYear,universityName,universityType,faculty,departmentName,idOSYM,programType,language,scholarshipRate,quota,...,totalStudentNumber,Urap_Rank,Urap_Score,normalized_department,normalized_faculty,Estimated Fee,avg_monthly_income_group,base_salary_by_year,inflation_by_year,growth_by_year
0,2024,İBN HALDUN ÜNİVERSİTESİ,vakıf,yönetimbilimlerifakültesi,i̇şletme,208610078,EA,İngilizce,100.0,8,...,861.0,112.0,492.920000,i̇şletme,yönetimbilimlerifakültesi,0.0,0.0,17002,64.0,2.7
1,2023,İBN HALDUN ÜNİVERSİTESİ,vakıf,yönetimbilimlerifakültesi,i̇şletme,208610078,EA,İngilizce,100.0,8,...,861.0,123.0,494.510000,i̇şletme,yönetimbilimlerifakültesi,0.0,0.0,11402,64.0,3.8
2,2022,İBN HALDUN ÜNİVERSİTESİ,vakıf,yönetimbilimlerifakültesi,i̇şletme,208610078,EA,İngilizce,100.0,10,...,861.0,145.0,392.554937,i̇şletme,yönetimbilimlerifakültesi,0.0,0.0,5500,64.0,5.4
3,2021,İBN HALDUN ÜNİVERSİTESİ,vakıf,yönetimbilimlerifakültesi,i̇şletme,208610078,EA,İngilizce,100.0,10,...,861.0,148.0,266.210000,i̇şletme,yönetimbilimlerifakültesi,0.0,0.0,2825,36.0,11.6
4,2020,İBN HALDUN ÜNİVERSİTESİ,vakıf,yönetimbilimlerifakültesi,i̇şletme,208610078,EA,İngilizce,100.0,10,...,861.0,136.0,260.440000,i̇şletme,yönetimbilimlerifakültesi,0.0,0.0,2103,14.0,1.8


In [15]:
# Copy the values from 'Estimated Fee' to 'tuitionFee'
df['tuitionFee'] = df['Estimated Fee']

# Drop the 'Estimated Fee' column
df.drop(columns=['Estimated Fee'], inplace=True)

# Display the updated DataFrame
# df.head()
print(df.isnull().sum())

academicYear                       0
universityName                     0
universityType                     0
faculty                            0
departmentName                     0
idOSYM                             0
programType                        0
language                           0
scholarshipRate                31241
quota                              0
occupiedSlots                      0
tuitionFee                     10152
universityFoundingYear         65281
facultyFoundingYear            65281
universityLocation                 0
universityRegion                   0
profCount                          0
assoCount                          0
docCount                           0
baseRanking                        0
topRanking                         0
avgAdmissionRanking(TYT)           0
baseAdmissionRanking(TYT)          0
stdDeviationStudents               0
revenue                        65281
outOfCityStudentRate               0
totalPreference                    0
t

In [16]:
non_numeric_columns = df.select_dtypes(exclude=['number']).columns
print("Non-numeric columns:", non_numeric_columns.tolist())

Non-numeric columns: ['universityName', 'universityType', 'faculty', 'departmentName', 'programType', 'language', 'universityLocation', 'universityRegion', 'top1AdmittedRatio', 'top3AdmittedRatio', 'top10AdmittedRatio', 'normalized_department', 'normalized_faculty']


In [17]:
# Fill missing values in the topRanking column with the mean
df['topRanking'] = df['topRanking'].fillna(
    df.groupby('universityName')['topRanking'].transform('mean')
)
# Calculate revenue for 'vakıf' university types
df.loc[df['universityType'] == 'vakıf', 'revenue'] = df['tuitionFee'] * df['occupiedSlots']

# Find and drop columns with all empty values
empty_columns = df.columns[df.isnull().all()]
df.drop(columns=empty_columns, inplace=True)
df.drop(columns=['normalized_faculty','normalized_department'], inplace=True)

# Calculate the tuition fee increase compared to the previous year
df.sort_values(by=['universityName', 'departmentName', 'academicYear'], inplace=True)

# Display the updated DataFrame
df.head()

,academicYear,universityName,universityType,faculty,departmentName,idOSYM,programType,language,scholarshipRate,quota,...,baseScore,topScore,totalForeignStudents,totalStudentNumber,Urap_Rank,Urap_Score,avg_monthly_income_group,base_salary_by_year,inflation_by_year,growth_by_year
17649,2019,ABDULLAH GÜL ÜNİVERSİTESİ,devlet,mühendislikfakültesi,bilgisayarmühendisliği,106510077,SAY,İngilizce,NaN,62,...,4.931221e+06,518.21984,235.0,2670.0,26.0,551.940000,2.0,2103,11.8,0.7
17648,2020,ABDULLAH GÜL ÜNİVERSİTESİ,devlet,mühendislikfakültesi,bilgisayarmühendisliği,106510077,SAY,İngilizce,NaN,72,...,2.339497e+02,279.88569,235.0,2670.0,36.0,521.630000,2.0,2103,14.0,1.8
17647,2021,ABDULLAH GÜL ÜNİVERSİTESİ,devlet,mühendislikfakültesi,bilgisayarmühendisliği,106510077,SAY,İngilizce,NaN,70,...,3.979366e+02,444.40481,235.0,2670.0,40.0,506.880000,2.0,2825,36.0,11.6
17646,2022,ABDULLAH GÜL ÜNİVERSİTESİ,devlet,mühendislikfakültesi,bilgisayarmühendisliği,106510077,SAY,İngilizce,NaN,75,...,4.694266e+02,513.57950,235.0,2670.0,43.0,716.794001,2.0,5500,64.0,5.4
17645,2023,ABDULLAH GÜL ÜNİVERSİTESİ,devlet,mühendislikfakültesi,bilgisayarmühendisliği,106510077,SAY,İngilizce,NaN,75,...,4.715531e+02,507.97368,235.0,2670.0,50.0,705.460000,2.0,11402,64.0,3.8


In [18]:
print(df.isnull().sum())

academicYear                       0
universityName                     0
universityType                     0
faculty                            0
departmentName                     0
idOSYM                             0
programType                        0
language                           0
scholarshipRate                31241
quota                              0
occupiedSlots                      0
tuitionFee                     10152
universityLocation                 0
universityRegion                   0
profCount                          0
assoCount                          0
docCount                           0
baseRanking                        0
topRanking                         0
avgAdmissionRanking(TYT)           0
baseAdmissionRanking(TYT)          0
stdDeviationStudents               0
revenue                        41991
outOfCityStudentRate               0
totalPreference                    0
top1PreferenceRatio                0
avgOrderofPreference               0
a

In [19]:
# Fill missing values in the topRanking column with the mean
df['topRanking'] = df['topRanking'].fillna(
    df.groupby('universityName')['topRanking'].transform('mean')
)
df['avgAdmissionRanking'] = pd.to_numeric(df['avgAdmissionRanking(TYT)'], errors='coerce')
df['baseAdmissionRanking'] = pd.to_numeric(df['baseAdmissionRanking(TYT)'], errors='coerce')
# Fill missing values in the avgAdmissionRanking column with the mean
df['avgAdmissionRanking'] = df['avgAdmissionRanking'].fillna(
    df.groupby('universityName')['avgAdmissionRanking'].transform('mean')
)
df['baseAdmissionRanking'] = df['baseAdmissionRanking'].fillna(
    df.groupby('universityName')['baseAdmissionRanking'].transform('mean')
)
df['stdDeviationStudents'] = df['stdDeviationStudents'].fillna(
    df.groupby('universityName')['stdDeviationStudents'].transform('mean')
)

df.drop(columns=['avgAdmissionRanking(TYT)', 'baseAdmissionRanking(TYT)'], inplace=True)

# Check for missing values in each column
df.fillna(0, inplace=True)

# ScholarshipRate and revenue columns have missing values devlet university types
missing_values = df.isnull().sum()
print("Missing values in each column:\n", missing_values)



Missing values in each column:
 academicYear                        0
universityName                      0
universityType                      0
faculty                             0
departmentName                      0
idOSYM                              0
programType                         0
language                            0
scholarshipRate                     0
quota                               0
occupiedSlots                       0
tuitionFee                          0
universityLocation                  0
universityRegion                    0
profCount                           0
assoCount                           0
docCount                            0
baseRanking                         0
topRanking                          0
stdDeviationStudents                0
revenue                             0
outOfCityStudentRate                0
sameRegionStudentRate               0
totalPreference                     0
top1PreferenceRatio                 0
top3PreferenceRati

Missing values imputated

In [20]:
non_numeric_columns = df.select_dtypes(exclude=['number']).columns

print("Non-numeric columns before conversion:", non_numeric_columns)


Non-numeric columns before conversion: Index(['universityName', 'universityType', 'faculty', 'departmentName',
       'programType', 'language', 'universityLocation', 'universityRegion',
       'outOfCityStudentRate', 'sameRegionStudentRate', 'top1AdmittedRatio',
       'top3AdmittedRatio', 'top10AdmittedRatio',
       'Time for Graduates to Find a Job', 'employment_rate'],
      dtype='object')


In [21]:
# Convert percentage columns to float
percentage_columns = ['top1AdmittedRatio', 'top3AdmittedRatio', 'top10AdmittedRatio']

for col in percentage_columns:
    df[col] = df[col].str.replace('%', '').str.replace(',', '.').astype(float) / 100

print("Non-numeric columns after conversion:", df.select_dtypes(exclude=['number']).columns)

Non-numeric columns after conversion: Index(['universityName', 'universityType', 'faculty', 'departmentName',
       'programType', 'language', 'universityLocation', 'universityRegion',
       'outOfCityStudentRate', 'sameRegionStudentRate',
       'Time for Graduates to Find a Job', 'employment_rate'],
      dtype='object')


In [22]:
# Convert dot columns to float UNKNOWNS to NaN values
dot_columns = ['sameRegionStudentRate', 'outOfCityStudentRate', 'Time for Graduates to Find a Job', 'employment_rate']

for col in dot_columns:
    df[col] = pd.to_numeric(df[col].str.replace(',', '.'), errors='coerce')


In [23]:
print("Non-numeric columns after conversion:", df.select_dtypes(exclude=['number']).columns)


Non-numeric columns after conversion: Index(['universityName', 'universityType', 'faculty', 'departmentName',
       'programType', 'language', 'universityLocation', 'universityRegion'],
      dtype='object')


In [24]:
print(df.isnull().sum())
df.rename(columns={'Time for Graduates to Find a Job': 'Time_for_employment'}, inplace=True)


academicYear                           0
universityName                         0
universityType                         0
faculty                                0
departmentName                         0
idOSYM                                 0
programType                            0
language                               0
scholarshipRate                        0
quota                                  0
occupiedSlots                          0
tuitionFee                             0
universityLocation                     0
universityRegion                       0
profCount                              0
assoCount                              0
docCount                               0
baseRanking                            0
topRanking                             0
stdDeviationStudents                   0
revenue                                0
outOfCityStudentRate                 182
sameRegionStudentRate                182
totalPreference                        0
top1PreferenceRa

In [25]:
df.fillna(0, inplace=True)
print(df.isnull().sum())
df.to_csv('final_dataset.csv', index=False)

academicYear                   0
universityName                 0
universityType                 0
faculty                        0
departmentName                 0
idOSYM                         0
programType                    0
language                       0
scholarshipRate                0
quota                          0
occupiedSlots                  0
tuitionFee                     0
universityLocation             0
universityRegion               0
profCount                      0
assoCount                      0
docCount                       0
baseRanking                    0
topRanking                     0
stdDeviationStudents           0
revenue                        0
outOfCityStudentRate           0
sameRegionStudentRate          0
totalPreference                0
top1PreferenceRatio            0
top3PreferenceRatio            0
top9PreferenceRatio            0
avgOrderofPreference           0
avgAdmittedStudentPrefOrder    0
top1AdmittedRatio              0
top3Admitt